# Pdt_Clf - Multinomial Naive Bayes

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import joblib

# Load Data

In [2]:
df = pd.read_parquet(r"input_data\ccd_labeled_train_validate.parquet")

print(df.shape)
display(df.head())                    

(13268, 6)


date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text product   action  \
0                      i am still waiting on my card    card  inquire   
1  what can i do if my card still hasnt arrived a...    card  inquire   
2  i have been waiting over a week is the card st...    card  inquire   
3  can i track my card while it is in the process...    card  inquire   
4  how do i know if i will get my card or if it i...    card  inquire   

  sentiment  
0  negative  
1  negative  
2  negative  
3   neutral  
4   neutral

In [3]:
# testing - check for duplicates or errors in labels
sorted(df['product'].unique())

['account',
 'atm',
 'card',
 'credit_card',
 'current_account',
 'debit_card',
 'fixed_deposit',
 'forex_account',
 'loan',
 'mobile_app',
 'others',
 'savings_account',
 'website']

In [4]:
# df['product'].value_counts()

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['product'], test_size=0.2, random_state=42, stratify=df['product'])

In [6]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Feature extraction using TF-IDF
    ('classifier', MultinomialNB())  # Classifier (Naive Bayes)
])
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('classifier', MultinomialNB())])

In [7]:
# Make predictions and evaluate the model
y_pred = pipeline.predict(X_test)

# Print evaluation metrics
print("Accuracy:", round(accuracy_score(y_test, y_pred), 3))
print(classification_report(y_test, y_pred))

Accuracy: 0.557
                 precision    recall  f1-score   support

        account       0.45      0.69      0.54       416
            atm       0.89      0.48      0.62       212
           card       0.52      0.96      0.67       730
    credit_card       0.71      0.05      0.08       111
current_account       0.90      0.30      0.46       151
     debit_card       0.80      0.04      0.07       114
  fixed_deposit       1.00      0.09      0.16        81
  forex_account       0.67      0.71      0.69       260
           loan       1.00      0.12      0.22        83
     mobile_app       0.81      0.24      0.37       163
         others       0.00      0.00      0.00        80
savings_account       0.64      0.41      0.50       164
        website       0.95      0.22      0.36        89

       accuracy                           0.56      2654
      macro avg       0.72      0.33      0.37      2654
   weighted avg       0.65      0.56      0.50      2654



C:\py312_env_IssPlp\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\py312_env_IssPlp\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\py312_env_IssPlp\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Predict with Trained Model

In [8]:
# Example usage of the trained model to make predictions on new data
new_texts = [
    "my card is stuck in the atm machine. I want it back.", 
    "Investing in stocks is risky.",
    "Why are your fixed deposit rates so terrible? I don't want to roll over my fixed deposit."
]
predictions = pipeline.predict(new_texts)

for text, prediction in zip(new_texts, predictions):
    print(f'Text: "{text}" => Predicted Keyword: "{prediction}"')

Text: "my card is stuck in the atm machine. I want it back." => Predicted Keyword: "atm"
Text: "Investing in stocks is risky." => Predicted Keyword: "card"
Text: "Why are your fixed deposit rates so terrible? I don't want to roll over my fixed deposit." => Predicted Keyword: "account"


# Save Model

In [9]:
# Save the model using joblib
joblib.dump(pipeline, 'trained_model/pdt_clr_MNB_model.pkl')

print("Model saved!")

Model saved!


# Reload Model and do Predict

In [10]:
del pipeline

In [11]:
# Load the model from the file
pipeline = joblib.load('trained_model/pdt_clr_MNB_model.pkl')

In [12]:
new_texts = [
    "my card is stuck in the atm machine. I want it back.", 
    "Investing in stocks is risky.",
    "Why are your fixed deposit rates so terrible? I don't want to roll over my fixed deposit."
]
predictions = pipeline.predict(new_texts)

for text, prediction in zip(new_texts, predictions):
    print(f'Text: "{text}" => Predicted Keyword: "{prediction}"')

Text: "my card is stuck in the atm machine. I want it back." => Predicted Keyword: "atm"
Text: "Investing in stocks is risky." => Predicted Keyword: "card"
Text: "Why are your fixed deposit rates so terrible? I don't want to roll over my fixed deposit." => Predicted Keyword: "account"
